# **SD Forge Neo + Zrok para Lightning.ai**


**1.1. Copia y pega lo siguiente en la terminal de JupyterLab (Lightning.ai)**<br>
Cuando aparezca el kernel `Python 3.11 (py311)`, reinicia desde `Kernel → Restart Kernel and Clear Outputs` y vuelve a elegirlo.


**1. Prepara y limpia el espacio de trabajo antes de instalar** (puedes ejecutarlo en CPU).


In [ ]:
!conda create -n py311 python=3.11 -y

!conda activate py311
!conda install ipykernel -y

!python -m ipykernel install --user \
       --name py311 \
       --display-name "Python 3.11 (py311)

Error: Conda create is not allowed. A Studio has a default conda environment (max 1 environment). Start a new Studio to create a new environment.
Error: Conda activate is not allowed. A Studio has a default conda environment (max 1 environment). Start a new Studio to create a new environment.
Retrieving notices: done
Channels:
 - defaults
Platform: linux-64
Solving environment: done

# All requested packages already installed.



In [2]:

from pathlib import Path
import os
import shutil

BASE_PATH = Path("/teamspace/studios/this_studio")
TMP_ROOT = Path("/tmp")
REPO_NAME = "sd-webui-forge-neo"
REPO_DIR = BASE_PATH / REPO_NAME

env_map = {
    "BASE_PATH": str(BASE_PATH),
    "REPO_DIR": str(REPO_DIR),
    "TMP_ROOT": str(TMP_ROOT),
    "TMP_MODELS": str(TMP_ROOT / "models"),
    "TMP_CONTROLNET": str(TMP_ROOT / "controlnet_models"),
    "HF_HOME": str(TMP_ROOT / "huggingface"),
    "TORCH_HOME": str(TMP_ROOT / "torch"),
    "TRANSFORMERS_CACHE": str(TMP_ROOT / "transformers"),
    "XDG_CACHE_HOME": str(TMP_ROOT / "xdg-cache"),
}
for key, value in env_map.items():
    os.environ[key] = value

def safe_remove(path: Path):
    try:
        if path.is_symlink() or path.is_file():
            path.unlink(missing_ok=True)
        elif path.exists() and path.is_dir():
            shutil.rmtree(path, ignore_errors=True)
    except FileNotFoundError:
        pass

for target in [REPO_DIR, BASE_PATH / "tmp"]:
    safe_remove(target)

for key in ("TMP_MODELS", "TMP_CONTROLNET", "HF_HOME", "TORCH_HOME", "TRANSFORMERS_CACHE", "XDG_CACHE_HOME"):
    folder = Path(os.environ[key])
    safe_remove(folder)
    folder.mkdir(parents=True, exist_ok=True)

symlink_target = BASE_PATH / "tmp"
if not symlink_target.exists():
    symlink_target.symlink_to(TMP_ROOT, target_is_directory=True)

print("Entorno inicializado:")
for key in ("BASE_PATH", "REPO_DIR", "TMP_MODELS", "TMP_CONTROLNET"):
    print(f"  {key} = {os.environ[key]}")


Entorno inicializado:
  BASE_PATH = /teamspace/studios/this_studio
  REPO_DIR = /teamspace/studios/this_studio/sd-webui-forge-neo
  TMP_MODELS = /tmp/models
  TMP_CONTROLNET = /tmp/controlnet_models


<h4><b>2. INSTALAR Y ACTUALIZAR PAQUETES</b></h4> (puedes ejecutarlo en CPU).


In [5]:

import shutil
import subprocess
import sys

def run_pip(*args):
    subprocess.run([sys.executable, "-m", "pip", *args], check=True)

run_pip("install", "--upgrade", "pip")
run_pip("install", "-q", "ipywidgets")

if shutil.which("conda"):
    subprocess.run(["conda", "update", "-n", "base", "-c", "defaults", "conda", "-y"], check=True)
    subprocess.run(["conda", "install", "-c", "conda-forge", "ipykernel", "-y"], check=True)
else:
    print("Conda no está disponible en Lightning.ai; se omite su actualización.")

print("Proceso terminado, puedes seguir con las demás celdas.")

!sudo apt install aria2 -q


Conda no está disponible en Lightning.ai; se omite su actualización.
Proceso terminado, puedes seguir con las demás celdas.
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libaria2-0 libcares2 libssh2-1t64
The following NEW packages will be installed:
  aria2 libaria2-0 libcares2 libssh2-1t64
0 upgraded, 4 newly installed, 0 to remove and 7 not upgraded.
Need to get 1686 kB of archives.
After this operation, 6117 kB of additional disk space will be used.
Get:1 https://us-east-1.ec2.archive.ubuntu.com/ubuntu noble/main amd64 libcares2 amd64 1.27.0-1.0ubuntu1 [73.7 kB]
Get:2 https://us-east-1.ec2.archive.ubuntu.com/ubuntu noble/main amd64 libssh2-1t64 amd64 1.11.0-4.1build2 [120 kB]
Get:3 https://us-east-1.ec2.archive.ubuntu.com/ubuntu noble/universe amd64 libaria2-0 amd64 1.37.0+debian-1build3 [1105 kB]
Get:4 https://us-east-1.ec2.archive.ubuntu.com/ubuntu noble/universe amd64 aria2 amd64 1.37.0+deb

<h4><b> 3. Instala Stable Diffusion Forge Neo (branch neo)</b></h4>


In [6]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from pathlib import Path
import os
import shutil
import subprocess
import sys

REPO_URL = "https://github.com/Haoming02/sd-webui-forge-classic.git"
REPO_BRANCH = "neo"
REPO_DIR = Path(os.environ["REPO_DIR"])
TMP_MODELS = Path(os.environ["TMP_MODELS"])
TMP_CONTROLNET = Path(os.environ["TMP_CONTROLNET"])

ext_list = [
    "gutris1/sd-hub",
    "zanllp/sd-webui-infinite-image-browsing",
    "Coyote-A/ultimate-upscale-for-automatic1111",
    "DominikDoom/a1111-sd-webui-tagcomplete",
    "Bing-su/adetailer",
    "NoCrypt/sd-fast-pnginfo",
    "BlafKing/sd-civitai-browser-plus",
    "viyiviyi/stable-diffusion-webui-zoomimage",
    "gutris1/sd-simple-dimension-preset",
    "pamparamm/sd-perturbed-attention",
    "anxety-solo/sd-civitai-browser-plus",
    "Bing-su/sd-webui-tunnels",
    "Haoming02/sd-forge-couple",
]

instala = widgets.Button(description="Instalar Forge Neo", button_style="info")
salida = widgets.Output()

def run(cmd):
    subprocess.run(cmd, check=True)

def instalar(_):
    with salida:
        salida.clear_output()
        print("⬇️  Clonando Forge Neo…")
        if REPO_DIR.exists():
            shutil.rmtree(REPO_DIR)
        run(["git", "clone", "--branch", REPO_BRANCH, "--single-branch", REPO_URL, str(REPO_DIR)])

        print("🔗 Configurando carpetas temporales…")
        models_dir = REPO_DIR / "models"
        stable_dir = models_dir / "Stable-diffusion"
        stable_dir.mkdir(parents=True, exist_ok=True)
        tmp_models_link = stable_dir / "tmp_models"
        if tmp_models_link.exists() or tmp_models_link.is_symlink():
            tmp_models_link.unlink()
        tmp_models_link.symlink_to(TMP_MODELS, target_is_directory=True)

        controlnet_dir = models_dir / "ControlNet"
        controlnet_dir.mkdir(parents=True, exist_ok=True)
        tmp_controlnet_link = controlnet_dir / "tmp_controlnet"
        if tmp_controlnet_link.exists() or tmp_controlnet_link.is_symlink():
            tmp_controlnet_link.unlink()
        tmp_controlnet_link.symlink_to(TMP_CONTROLNET, target_is_directory=True)

        print("📦 Descargando modelos ESRGAN…")
        esrgan_dir = models_dir / "ESRGAN"
        esrgan_dir.mkdir(parents=True, exist_ok=True)
        run(["curl", "-Lo", str(esrgan_dir / "4x-UltraSharp.pth"), "https://huggingface.co/Kim2091/AnimeSharp/resolve/main/4x-AnimeSharp.pth"])
        run(["curl", "-Lo", str(esrgan_dir / "4x-AnimeSharp.pth"), "https://huggingface.co/utnah/esrgan/resolve/main/4x-AnimeSharp.pth"])

        print("🧩 Instalando extensiones…")
        ext_dir = REPO_DIR / "extensions"
        ext_dir.mkdir(parents=True, exist_ok=True)
        for ext in ext_list:
            destino = ext_dir / ext.split("/")[1]
            if destino.exists():
                shutil.rmtree(destino)
            run(["git", "clone", "--depth", "1", f"https://github.com/{ext}", str(destino)])

        print("📥 Instalando utilidades (gdown, aria2)…")
        subprocess.run([sys.executable, "-m", "pip", "install", "-q", "gdown", "aria2"], check=True)
        print("✅ Instalación completada.")

instala.on_click(instalar)
display(instala, salida)


Button(button_style='info', description='Instalar Forge Neo', style=ButtonStyle())

Output()

<h4><b> 4. Descarga checkpoints al almacenamiento temporal</b></h4>


<h4><b>MODELOS NOOBAI-ILLUSTRIOUS</b></h4>

In [7]:

import ipywidgets as widgets
from IPython.display import display, clear_output
from pathlib import Path
import os

TMP_MODELS = Path(os.environ["TMP_MODELS"])
REPO_DIR = Path(os.environ["REPO_DIR"])
stable_dir = REPO_DIR / "models" / "Stable-diffusion"
stable_dir.mkdir(parents=True, exist_ok=True)
tmp_link = stable_dir / "tmp_models"
if not tmp_link.exists():
    tmp_link.symlink_to(TMP_MODELS, target_is_directory=True)

modelos = {
    "V-pred – waiSHUFFLENOOBvpred v2.0": (
        "https://huggingface.co/WhiteAiZ/waiSHUFFLENOOB_v20/resolve/main/waiSHUFFLENOOB_vPred20.safetensors",
        "waiSHUFFLENOOB_vPred20.safetensors"
    ),
    "V-pred – PersonalMerge vp05": (
        "https://huggingface.co/datasets/John6666/model-mirror-14/resolve/main/illustriousXLPersonalMerge_vp05testLowsteps.safetensors",
        "PersonalMerge_vp05testLowsteps.safetensors"
    ),
    "V-pred – Rouwei_v080Vpred.safetensors": (
        "https://huggingface.co/WhiteAiZ/RouWei/resolve/main/rouwei_v080Vpred.safetensors",
        "rouwei_v080Vpred.safetensors"
    ),
    "EPS – WAI V15": (
        "https://huggingface.co/WhiteAiZ/WAI-NSFW-illustrious-SDXL-V015/resolve/main/waiNSFWIllustrious_v150.safetensors",
        "waiNSFWIllustrious_v150.safetensors"
    ),
    "EPS – ntrMIXIllustriousXL": (
        "https://huggingface.co/misri/ntrMIXIllustriousXL_xiii/resolve/main/ntrMIXIllustriousXL_xiii.safetensors",
        "ntrMIXIllustriousXL_xiii.safetensors"
    ),
    "EPS – ikastrious": (
        "https://huggingface.co/wsj1995/Checkpoint/resolve/main/874216/2002320/ikastrious_v132bStable.safetensors",
        "ikastrious_v132bStable.safetensors"
    ),
    "EPS – waiSHUFFLENOOB v2.0": (
        "https://huggingface.co/WhiteAiZ/waiSHUFFLENOOB_v20/resolve/main/waiSHUFFLENOOB_v20.safetensors",
        "waiSHUFFLENOOB_v20.safetensors"
    ),
}

dropdown = widgets.Dropdown(
    options=list(modelos.keys()),
    description='Modelo:',
    style={'description_width': 'initial'},
    layout={'width': 'max-content'}
)
boton = widgets.Button(description='Descargar', button_style='info')
salida = widgets.Output()

def descargar(_):
    url, nombre = modelos[dropdown.value]
    destino = TMP_MODELS / nombre
    with salida:
        clear_output()
        print(f'⬇️  Descargando {nombre} …')
        get_ipython().system(
            f'aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{url}" -o "{destino}"'
        )
        print(f'✅ Guardado en {destino}')

boton.on_click(descargar)
display(widgets.HBox([dropdown, boton]), salida)


Output()

<h4><b>Descargar Modelo Opcional</b></h4>

In [ ]:
<h4><b> 5. ControlNet temporales</b></h4>


**Descargar controlnet union pro max**

In [ ]:

from pathlib import Path
import os
import subprocess

REPO_DIR = Path(os.environ["REPO_DIR"])
TMP_CONTROLNET = Path(os.environ["TMP_CONTROLNET"])
controlnet_dir = REPO_DIR / "models" / "ControlNet"
controlnet_dir.mkdir(parents=True, exist_ok=True)
controlnet_link = controlnet_dir / "tmp_controlnet"
if not controlnet_link.exists():
    controlnet_link.symlink_to(TMP_CONTROLNET, target_is_directory=True)

subprocess.run([
    "aria2c", "--console-log-level=error", "-c", "-x", "16", "-s", "16", "-k", "1M",
    "https://huggingface.co/xinsir/controlnet-union-sdxl-1.0/resolve/main/diffusion_pytorch_model_promax.safetensors",
    "-d", str(TMP_CONTROLNET), "-o", "controlnet_union_promax.safetensors"
], check=True)
print('DESCARGAS TERMINADAS')


<h4><b> Revisa el espacio disponible</b></h4>

In [ ]:
<h4><b>Ver todas las carpetas y su tamaño</b></h4>


<h4><b>Ver todas las carpetas y su tamaño</b></h4>

In [ ]:
!du -h --max-depth=1 "$REPO_DIR" | sort -hr


<h4><b>Eliminar cache</b></h4> (hazlo después de cada instalación).


In [7]:

from pathlib import Path
import os
import shutil

cache_dir = Path(os.environ["BASE_PATH"]) / '.cache'
if cache_dir.exists():
    shutil.rmtree(cache_dir, ignore_errors=True)
    print(f'Se eliminó {cache_dir}')
else:
    print('No hay cache que eliminar.')


Se eliminó /teamspace/studios/this_studio/.cache


<h4><b>Liberar espacio</b></h4> (hazlo después de cada instalación nueva).


In [ ]:

import shutil
import subprocess

if shutil.which('conda'):
    subprocess.run(['conda', 'clean', '--all', '-y'], check=True)
else:
    print('Conda no está disponible; omitiendo limpieza con conda.')


<h4><b>Borrar y limpiar la carpeta outputs.</b></h4>


In [ ]:

from pathlib import Path
import shutil

outputs_dir = Path(os.environ["REPO_DIR"]) / 'outputs'
if outputs_dir.exists():
    shutil.rmtree(outputs_dir, ignore_errors=True)
    outputs_dir.mkdir(parents=True, exist_ok=True)
    print('Outputs limpiados.')
else:
    print('La carpeta outputs aún no existe.')


<h5><b>INSTALACION DE ZROK</b></h5>

No olvide pegar su token NGROK a continuación. Crea un token [aquí](https://dashboard.ngrok.com/get-started/your-authtoken)

In [8]:
%cd /tmp
!wget https://github.com/openziti/zrok/releases/download/v1.1.10/zrok_1.1.10_linux_amd64.tar.gz -q
!mkdir /tmp/zrok
!tar -xf ./zrok*linux*.tar.gz -C /tmp/zrok
!mkdir -p /tmp/zrok/bin && install /tmp/zrok/zrok /tmp/zrok/bin
import os
os.environ['PATH'] = "/tmp/zrok/bin:" + os.environ['PATH']
!zrok version

/tmp
               _    
 _____ __ ___ | | __
|_  / '__/ _ \| |/ /
 / /| | | (_) |   < 
/___|_|  \___/|_|\_\

v1.1.10 [74eb6fc5]



#Usar solamente en la terminal para crear cuenta en zrok
**export PATH=$PATH:/tmp/zrok/bin <p>**
**zrok invite**

<h4><b> Activa o desactiva tu token de zrok, usar "enable" + tu token para activar y "disable" sin el token para desactivar</b></h4>

In [9]:
!zrok enable rouDjM78m7Hi

[ERROR] you already have an enabled environment, zrok disable first before you zrok enable


<h5><b>LANZAR SD CON ZROK</b></h5>


In [14]:

%cd /teamspace/studios/this_studio/sd-webui-forge-neo
from multiprocessing import Process
import time

def run_command1():
    time.sleep(15)
    !zrok share public http://localhost:7860 --headless

def run_command2():
    time.sleep(2)
    !pip install torch==2.7.1
    !python launch.py --no-hashing  --disable-console-progressbars --xformers --cuda-stream --always-high-vram --enable-insecure-extension-access --theme dark --listen --port 7860 --share

if __name__ == '__main__':
    p1 = Process(target=run_command1)
    p2 = Process(target=run_command2)
    p1.start()
    p2.start()
    p1.join()
    p2.join()

/teamspace/studios/this_studio/sd-webui-forge-neo
  Using cached torch-2.7.1-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (29 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.6.77-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.6.77-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.6.80-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.5.1.17-py3-none-manylinux_2_28_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.6.4.1-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.3.0.4-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.7.77-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.7.1.2-py3-none-manylinux2014_x86_64.manylin

KeyboardInterrupt: 

^C
Interrupted with signal 2 in <frame at 0x7004c81318c0, file '/teamspace/studios/this_studio/sd-webui-forge-neo/modules_forge/main_thread.py', line 43, code loop>
^C


<h5><b>LANZAR SD CON NGROK</b></h5>


In [ ]:

# No te olvides de pegar tu token de NGROK aquí
%cd $REPO_DIR
import subprocess
import sys

token = 'coloca_tu_token_aqui'  # reemplázalo por tu token real
if token == 'coloca_tu_token_aqui':
    raise ValueError('Reemplaza token por tu token de NGROK antes de ejecutar.')
subprocess.run([
    sys.executable, 'launch.py', '--ngrok', token,
    '--no-hashing', '--disable-console-progressbars', '--disable-xformers',
    '--opt-sdp-attention', '--enable-insecure-extension-access',
    '--theme', 'dark', '--gradio-queue', '--no-download-sd-model'
], check=True)


<h4><b>🖼️ Descargar carpeta Output</b></h4>


In [ ]:

# download and compress all outputs
import os
import zipfile
from pathlib import Path

outputs_dir = Path(os.environ["REPO_DIR"]) / 'outputs'
destination = Path('outputs')
destination.mkdir(parents=True, exist_ok=True)

def zip_folder(input_folder, output_folder, max_size_mb=199):
    input_folder = Path(input_folder)
    output_folder = Path(output_folder)
    zip_number = 1
    current_zip_name = output_folder / f'part_{zip_number}.zip'
    current_zip = zipfile.ZipFile(current_zip_name, 'w', zipfile.ZIP_DEFLATED)
    current_zip_size = 0

    for root, _, files in os.walk(input_folder):
        for file in files:
            file_path = Path(root) / file
            file_size = file_path.stat().st_size
            if current_zip_size + file_size > max_size_mb * 1024 * 1024:
                current_zip.close()
                zip_number += 1
                current_zip_name = output_folder / f'part_{zip_number}.zip'
                current_zip = zipfile.ZipFile(current_zip_name, 'w', zipfile.ZIP_DEFLATED)
                current_zip_size = 0
            current_zip.write(file_path, file_path.relative_to(input_folder))
            current_zip_size += file_size

    current_zip.close()

if outputs_dir.exists():
    zip_folder(outputs_dir, destination)
    print(f'Carpeta outputs comprimida en {destination.resolve()}')
else:
    print('La carpeta outputs aún no existe.')
